# 1: Slab model
---------------

This notebook computes an infrared spectrum following a slab model with a black body as central star.

_This exercise was originally written by Michel Hillen in 2015, following the original IDL code of Tijl Verhoelst, then modified by Ana Escorza in 2016, and finally by Frederik De Ceuster in 2024._

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from astropy import units
from astropy import constants
from astropy.modeling.models import BlackBody
from astropy.visualization import quantity_support

Choose the model parameters: the temperatures and angular sizes of the star and the slab, as well as the column density of the slab

In [ ]:
# Define the model parameters
temperature_star = 0
temperature_slab = 0
theta_star = 0
theta_slab = 0
column_density = 0

Now read the water cross section file for the chosen slab temperature (note it must be between 500 and 2500 K in multiples of 100)

In [ ]:
# Read cross section file
cross_section_file = 'data/H2O_tau_{:04d}K.txt'.format(temperature_slab)
wavelength, cross_section = np.loadtxt(cross_section_file,
                                       skiprows=1,
                                       unpack=True)

# Attach astropy units
wavelength *= units.nm
cross_section *= units.cm**2


In [ ]:
# Attach units to the model parameters
temperature_star *= units.K
temperature_slab *= units.K
theta_star *= units.mas
theta_slab *= units.mas
column_density *= units.cm**(-2)

In [ ]:
def convolve_with_gaussian(x, y, width):
    # Convert from the FWHM to the constant in the Gaussian
    width = width / (2.0 * np.sqrt(2.0 * np.log(2.0)))

    # Create the Gaussian kernel
    n = len(x)
    width_pix = width / ((x[n - 1] - x[0]) / n)
    nker = 20 * np.floor(width_pix)
    z = (np.arange(nker) - np.floor(nker / 2)) / width_pix
    kernel = np.exp(-0.5 * z**2)

    # Normalize it for flux conservation
    normalisation_factor = kernel.sum()

    # Convolve
    result = (1.0 / normalisation_factor) * np.convolve(y, kernel, mode='same')

    return result

Optical depth, assuming a constant opacity and density along the line-of-sight, is
\begin{equation}
    \tau_{\nu} \ = \ \kappa_{\nu} \, \rho \, d = \ N \, \sigma_{\nu}
\end{equation}
Solution of the radiative transfer equation, assuming a constant source function, $S_{\nu}$, along the line-of-sight,
\begin{equation}
    I_{\nu} \ = \ I^{\text{bdy}}_{\nu} \, e^{-\tau_{\nu}} \ + \ S_{\nu} \left( 1 - e^{-\tau_{\nu}} \right) .
\end{equation}

In [ ]:
# Define the star and slab as black body radiators at a given temperature
bb_star = BlackBody(temperature=temperature_star)
bb_slab = BlackBody(temperature=temperature_slab)

# Optical depth through slab (assuming a constant density and cross section along the line-of-sight)
optical_depth = cross_section * column_density

# Spectrum of star
spec_star = bb_star(wavelength) * np.pi * theta_star**2

# Spectrum of star after trasfer through slab
spec_slab =   bb_star(wavelength) * np.pi * theta_star**2 *      np.exp(-optical_depth) \
            + bb_slab(wavelength) * np.pi * theta_slab**2 * (1 - np.exp(-optical_depth))

# Spectrum convolved with Gaussian beam
spec_conv = convolve_with_gaussian(wavelength, spec_slab, width=3.0 * units.nm)


In [ ]:
# Plot the stellar and slab spectra (note the first and last 10 points of the convolved slab spectrum are affected by boundary issues and are removed)
with quantity_support():
    plt.figure(dpi=130)
    plt.plot(wavelength.to(units.micron),
             spec_star.to(units.Jansky),
             label='star')
    plt.plot(wavelength.to(units.micron),
             spec_slab.to(units.Jansky),
             label='star+slab')
    plt.plot(wavelength[10:-10].to(units.micron),
             spec_conv[10:-10].to(units.Jansky),
             label='convolved low res')
    plt.xscale('log')
    plt.title(
        f"T_st={temperature_star}, T_sl={temperature_slab}, t_st={theta_star}, t_sl={theta_slab}, \n N={column_density}"
    )
    plt.legend()
    try:
        plt.savefig(
            f'plots/Tst_{temperature_star.value}_Tsl_{temperature_slab.value}_tst_{theta_star.value}_tsl_{theta_slab.value}_N_{column_density.value}.png'
        )
    except FileNotFoundError:
        os.mkdir('plots')
        plt.savefig(
            f'plots/Tst_{temperature_star.value}_Tsl_{temperature_slab.value}_tst_{theta_star.value}_tsl_{theta_slab.value}_N_{column_density.value}.png'
        )